# Regularization

Let's improve our understanding of what impacted **Titanic** passengers' chance of survival
- We will use logistic classifiers which are easy to interpret
- Remember we already did it with statsmodels in lecture "Decision Science - Logistic Regression"
- We were using `p-values` & statistical assumptions to detect which features were irrelevant / don't generalize
- This time, we will use `regularization` to detect relevant/irrelevant features based on under/overfitting criteria
- **Our goal is to compare `L1` and `L2` penalties**

## 1. We load and preprocess the data for you

In [57]:
import pandas as pd
import numpy as np

In [58]:
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/ML_titanic_dataset_encoded.csv")

# the dataset is already one-hot-encoded
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [59]:
# We build X and y

y = data["survived"]
X = data.drop(columns="survived")
X.head()

,pclass,age,sibsp,parch,fare,sex_female,class_First,class_Third,who_child,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
0,3,22.0,1,0,7.2500,0,0,1,0,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,1,0,0,0,1


In [60]:
# We MinMaxScale our features for you
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X.shape

(714, 12)

## 1.  Logistic Regression without regularization

❓ Rank the features by decreasing order of importance after training a simple **non-regularized** Logistic Regression (i.e. look at the coefficients after fitting)
- Careful, `LogisticRegression` is penalized by default
- Increase `max_iter` to a larger number until model converges

In [61]:
# YOUR CODE HERE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(penalty='none', max_iter=1000)
logreg.fit(X_scaled, y)

coefficients = logreg.coef_[0]

feature_importance = {feature: coefficient for feature, coefficient in zip(X_scaled, coefficients)}
feature_importance
#sorted_feature_importance = sorted(feature_importance.items(), key=lambda x: abs(x[1]), reverse=True)

#for feature, importance in sorted_feature_importance:
   # print(f"{feature}: {importance}")

{'pclass': 2.5471874598469015,
 'age': -2.196151364913655,
 'sibsp': -2.4771314726291904,
 'parch': -0.8938204025266032,
 'fare': 1.3588116683194615,
 'sex_female': 2.671882643337724,
 'class_First': 2.3604172865126674,
 'class_Third': -2.456891197851845,
 'who_child': 1.3363564093558427,
 'embark_town_Cherbourg': -11.221670908941213,
 'embark_town_Queenstown': -11.918725145360382,
 'embark_town_Southampton': -11.523125628244285}

❓How do you interpret, in plain English, the value for the coefficient `sex_female`?

<details>
    <summary>Answer</summary>

> "All other things being equal (such as age, ticket class etc...),
being a women increases your log-odds of survival by 2.67 (your coef value)"
    
> "Controling for all other explaining factors available in this dataset,
being a women increases your odds of survival by exp(2.67) = 14"

</details>


In [62]:
# sex_female indicates the impact of gender on the survival outcome, while controlling for other factors available in the dataset

❓ What is the feature that most impacts the chances of survival according to your model?  
Fill the `top_1_feature` list below with the name of this feature

In [63]:
top_1_feature = ["embark_town_Queenstown"]

In [64]:
from nbresult import ChallengeResult
result = ChallengeResult('unregularized', top_1_feature = top_1_feature)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/saikotdasjoy/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/saikotdasjoy/code/Saikot1997/data-regularization/tests
plugins: asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_unregularized.py::TestUnregularized::test_top_1 PASSED              [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/unregularized.pickle

git commit -m 'Completed unregularized step'

git push origin master



## 2.  Logistic Regression with an L2 penalty

Let's use a **Logistic model** whose log-loss has been penalized with a **L2** term to figure out the **most important features** without overfitting.  
This is the "classification" equivalent to the "Ridge" regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance (look at the coefficients)
- By "strongly regularized" we mean "more than Sklearn's default regularization factor". 
- Sklearn's default values are very useful orders of magnitudes to keep in mind for "scaled features"

In [65]:
# YOUR CODE HERE
logreg_l2 = LogisticRegression(penalty='l2', C=0.1)
logreg_l2.fit(X_scaled, y)
coefficients_l2 = logreg_l2.coef_[0]
feature_importance_l2 = {feature: coefficient for feature, coefficient in zip(X.columns, coefficients_l2)}
sorted_feature_importance_l2 = sorted(feature_importance_l2.items(), key=lambda x: abs(x[1]), reverse=True)
for feature, importance in sorted_feature_importance_l2:
    print(f"{feature}: {importance}")

sex_female: 1.808600179157239
class_Third: -0.6369316496210842
who_child: 0.6028093829650082
pclass: -0.5392275413282104
age: -0.47771965781612646
class_First: 0.4414864652220872
sibsp: -0.34087206787183544
embark_town_Cherbourg: 0.25295572232682667
embark_town_Southampton: -0.1544026636226507
fare: 0.13679020187718863
embark_town_Queenstown: -0.13244336836632442
parch: -0.05390559890851231


❓ What are the top 2 features driving chances of survival according to your model?  
Fill the `top_2_features` list below with the name of these features

In [66]:
top_2_features = ['sex_female', 'class_Third']

#### 🧪 Test your code below

In [67]:
from nbresult import ChallengeResult
result = ChallengeResult('ridge', top_2 = top_2_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/saikotdasjoy/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/saikotdasjoy/code/Saikot1997/data-regularization/tests
plugins: asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_ridge.py::TestRidge::test_top2 PASSED                               [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/ridge.pickle

git commit -m 'Completed ridge step'

git push origin master



## 2. Logistic Regression with an L1 penalty

This time, we'll use a logistic model whose log-loss has been penalized with a **L1** term to **filter-out the less important features**.  
This is the "classification" equivalent to the **Lasso** regressor

❓ Instantiate a **strongly regularized** `LogisticRegression` and rank its features by importance

In [68]:
# YOUR CODE HERE
logreg_l1 = LogisticRegression(penalty='l1', C=0.1,solver='liblinear')

logreg_l1.fit(X_scaled, y)
coefficients_l1 = logreg_l1.coef_[0]
feature_importance_l1 = {feature: coefficient for feature, coefficient in zip(X.columns, coefficients_l1)}
sorted_feature_importance_l1 = sorted(feature_importance_l1.items(), key=lambda x: abs(x[1]), reverse=True)
for feature, importance in sorted_feature_importance_l1:
    print(f"{feature}: {importance}")

sex_female: 2.000478895874692
pclass: -1.4422022606145517
who_child: 0.25579734481558
embark_town_Southampton: -0.23932197320575513
class_Third: -0.1449159864021855
age: -0.06310032050197088
sibsp: 0.0
parch: 0.0
fare: 0.0
class_First: 0.0
embark_town_Cherbourg: 0.0
embark_town_Queenstown: 0.0


❓ What are the features that have absolutely no impact on chances of survival, according to your L1 model?  
Fill the `zero_impact_features` list below with the name of these features; you may have to add elements to the list.

- Do you notice how some of them were "highly important" according to the non-regularized model? 
- From now on, we will always regularize our linear models!

In [69]:
zero_impact_features = ["sibsp", "parch", "fare", "class_First", "embark_town_Cherbourg", "embark_town_Queenstown"]


#### 🧪 Test your code below

In [70]:
from nbresult import ChallengeResult
result = ChallengeResult('lasso', zero_impact_features = zero_impact_features)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /home/saikotdasjoy/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/saikotdasjoy/code/Saikot1997/data-regularization/tests
plugins: asyncio-0.19.0, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 1 item

test_lasso.py::TestLasso::test_zero_impact PASSED                        [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/lasso.pickle

git commit -m 'Completed lasso step'

git push origin master



**🏁 Congratulation! Don't forget to commit and push your notebook**